# **Importar Bibliotecas**

In [ ]:
import cvxpy as cp
import numpy as np
from sklearn.datasets import make_blobs
from sklearn.model_selection import train_test_split
from cvxopt import matrix as cvxopt_matrix
from cvxopt import solvers as cvxopt_solvers
import matplotlib.pyplot as plt
from sklearn.svm import SVC
from sklearn.metrics import accuracy_score
from sklearn.inspection import DecisionBoundaryDisplay

# **Criação e Plot dos Conjuntos de Dados Utilizados**

In [ ]:
X1, d1 = make_blobs(n_samples=20, centers=2, n_features=2,random_state=0)
d1 = 2*d1-1

X2, d2 = make_blobs(n_samples=100, centers=2, cluster_std =4 , n_features=2,random_state=17)
d2 = 2*np.mod(d2,2) -1

fig = plt.figure()
plt.scatter(X1[:, 0], X1[:, 1], c=d1)
plt.xlabel('A1')
plt.ylabel('B1')
plt.title('Primeiro Conjunto de Dados')
plt.legend()
plt.show()

fig = plt.figure()
plt.scatter(X2[:, 0], X2[:, 1], c=d2)
plt.xlabel('A2')
plt.ylabel('B2')
plt.title('Segundo Conjunto de Dados')
plt.legend()
plt.show()

# **Problema Primal - Hard Margin**

In [ ]:
# Resolução do Problema de Otimização
X1, d1 = make_blobs(n_samples=20, centers=2, n_features=2,random_state=0)
d1 = 2*d1-1

X2, d2 = make_blobs(n_samples=100, centers=2, cluster_std =4 , n_features=2,random_state=17)
d2 = 2*np.mod(d2,2) -1

def primalhm(X,d):
  m,n = X.shape

  W = cp.Variable(n)
  b = cp.Variable()
  P = np.ones((n, n))
  G = cp.multiply(d ,X @ W + np.ones(m)*b)
  h = np.ones(m)

  prob = cp.Problem(cp.Minimize((1/2)*cp.quad_form(W, P)),
                 [G >= h])
  prob.solve()

  w = np.ravel(W.value).reshape(-1,1)
  B = np.ravel(b.value)
  classificacao = np.sign(np.dot(X, w) + B)

  return W.value, b.value, classificacao

A1 = primalhm(X1,d1)
print(A1)

#Resultou em None para o seguinte caso:
#A2 = primalhm(X2,d2)
#print(A2)

In [ ]:
# Plot
X1, d1 = make_blobs(n_samples=20, centers=2, n_features=2,random_state=0)
d1 = 2*d1-1

X2, d2 = make_blobs(n_samples=100, centers=2, cluster_std =4 , n_features=2,random_state=17)
d2 = 2*np.mod(d2,2) -1

fig = plt.figure()
w = A1[0]; b = A1[1]
plt.scatter(X1[:, 0], X1[:, 1], c=d1)
plt.plot(X1[:,0], - (w[0]/w[1])*X1[:,0] - b/w[1], color = 'red', label='Hiperplano Ótimo')
plt.plot(X1[:,0], - (w[0]/w[1])*X1[:,0] - b/w[1] - 1/w[1], color = 'blue', label='Vetor Suporte')
plt.plot(X1[:,0], - (w[0]/w[1])*X1[:,0] - b/w[1] + 1/w[1], color = 'green', label='Vetor Suporte')
plt.xlabel('A1')
plt.ylabel('B1')
plt.title('Hiperplano ótimo encontrado através do problema primal para o primeiro conjunto de dados')
plt.legend()
plt.show()


# **Problema Primal - Soft Margin**

In [ ]:
# Resolução do Problema de Otimização
X1, d1 = make_blobs(n_samples=20, centers=2, n_features=2,random_state=0)
d1 = 2*d1-1

X2, d2 = make_blobs(n_samples=100, centers=2, cluster_std =4 , n_features=2,random_state=17)
d2 = 2*np.mod(d2,2) -1

def primalsm(X,d):
  c = 10

  m,n = X.shape

  W = cp.Variable(n)
  b = cp.Variable()
  Psi = cp.Variable(m)
  P = np.ones((n, n))
  G = cp.multiply(d ,X @ W + np.ones(m)*b)
  h = np.ones(m)

  prob = cp.Problem(cp.Minimize((1/2)*cp.quad_form(W, P) + c*cp.sum(Psi)),
                 [G >= h - Psi,
                  Psi >= 0])
  prob.solve()

  w = np.ravel(W.value).reshape(-1,1)
  B = np.ravel(b.value)
  classificacao = np.sign(np.dot(X, w) + B)

  return W.value, b.value, classificacao

A3 = primalsm(X1,d1)
A4 = primalsm(X2,d2)

print(A3)
print(A4)

In [ ]:
# Plot
X1, d1 = make_blobs(n_samples=20, centers=2, n_features=2,random_state=0)
d1 = 2*d1-1

X2, d2 = make_blobs(n_samples=100, centers=2, cluster_std =4 , n_features=2,random_state=17)
d2 = 2*np.mod(d2,2) -1

fig = plt.figure()
w = A3[0]; b = A3[1]
plt.scatter(X1[:, 0], X1[:, 1], c=d1)
plt.plot(X1[:,0], - (w[0]/w[1])*X1[:,0] - b/w[1], color = 'red', label='Hiperplano Ótimo')
plt.plot(X1[:,0], - (w[0]/w[1])*X1[:,0] - b/w[1] - 1/w[1], color = 'blue', label='Vetor Suporte')
plt.plot(X1[:,0], - (w[0]/w[1])*X1[:,0] - b/w[1] + 1/w[1], color = 'green', label='Vetor Suporte')
plt.xlabel('A1')
plt.ylabel('B1')
plt.title('Hiperplano ótimo encontrado através do problema primal com variável de folga para o primeiro conjunto de dados')
plt.legend()
plt.show()

# **Problema Dual - Hard Margin**

In [ ]:
# Resolução do Problema de Otimização
X1, d1 = make_blobs(n_samples=20, centers=2, n_features=2,random_state=0)
d1 = 2*d1-1

X2, d2 = make_blobs(n_samples=100, centers=2, cluster_std =4 , n_features=2,random_state=17)
d2 = 2*np.mod(d2,2) -1

def dualhm(X,d):
  m,n = X.shape
  d = d.reshape(-1,1) * 1.0
  Xd = d * X
  H = np.dot(Xd , Xd.T) * 1.0

  P = cvxopt_matrix(H)
  q = cvxopt_matrix(-np.ones((m, 1)))
  G = cvxopt_matrix(-np.eye(m))
  h = cvxopt_matrix(np.zeros(m))
  A = cvxopt_matrix(d.reshape(1, -1))
  b = cvxopt_matrix(np.zeros(1))

  sol = cvxopt_solvers.qp(P, q, G, h, A, b)
  alphas = np.array(sol['x'])

  W = ((d * alphas).T @ X).reshape(-1,1)

  S = (alphas > 1e-4).flatten()

  b = d[S] - np.dot(X[S], W)

  classificacao = np.sign(np.dot(X, W) + b[0])

  return W, b[0], classificacao

A5= dualhm(X1,d1)
A6= dualhm(X2,d2)

print(A5)
print(A6)

In [ ]:
# Plot
X1, d1 = make_blobs(n_samples=20, centers=2, n_features=2,random_state=0)
d1 = 2*d1-1

X2, d2 = make_blobs(n_samples=100, centers=2, cluster_std =4 , n_features=2,random_state=17)
d2 = 2*np.mod(d2,2) -1

fig = plt.figure()
w = A5[0]; b = A5[1]
plt.scatter(X1[:, 0], X1[:, 1], c=d1)
plt.plot(X1[:,0], - (w[0]/w[1])*X1[:,0] - b/w[1], color = 'red', label = 'Hiperplano Ótimo')
plt.plot(X1[:,0], - (w[0]/w[1])*X1[:,0] - b/w[1] - 1/w[1], color = 'blue', label = 'Vetor Suporte')
plt.plot(X1[:,0], - (w[0]/w[1])*X1[:,0] - b/w[1] + 1/w[1], color = 'green', label = 'Vetor Suporte')
plt.xlabel('A1')
plt.ylabel('B1')
plt.title('Hiperplano ótimo encontrado através do problema dual para o primeiro conjunto de dados')
plt.legend()
plt.show()

# **Problema Dual - Soft Margin**

In [ ]:
# Resolução do Problema de Otimização
X1, d1 = make_blobs(n_samples=20, centers=2, n_features=2,random_state=0)
d1 = 2*d1-1

X2, d2 = make_blobs(n_samples=100, centers=2, cluster_std =4 , n_features=2,random_state=17)
d2 = 2*np.mod(d2,2) -1

def dualsm(X,d):
  c = 10
  m,n = X.shape
  d = d.reshape(-1,1) * 1.0
  Xd = d * X
  H = np.dot(Xd , Xd.T) * 1.0

  P = cvxopt_matrix(H)
  q = cvxopt_matrix(-np.ones((m, 1)))
  G = cvxopt_matrix(np.vstack((np.eye(m)*-1,np.eye(m))))
  h = cvxopt_matrix(np.hstack((np.zeros(m), np.ones(m) * c/m)))
  A = cvxopt_matrix(d.reshape(1, -1))
  b = cvxopt_matrix(np.zeros(1))

  sol = cvxopt_solvers.qp(P, q, G, h, A, b)
  alphas = np.array(sol['x'])

  W = ((d * alphas).T @ X).reshape(-1,1)
  S = (alphas > 1e-4).flatten()
  b = d[S] - np.dot(X[S], W)

  classificacao = np.sign(np.dot(X, W) + b[0])

  return W, b[0], classificacao

A7= dualsm(X1,d1)
A8= dualsm(X2,d2)

print(A7)
print(A8)

In [ ]:
# Plot
X1, d1 = make_blobs(n_samples=20, centers=2, n_features=2,random_state=0)
d1 = 2*d1-1

X2, d2 = make_blobs(n_samples=100, centers=2, cluster_std =4 , n_features=2,random_state=17)
d2 = 2*np.mod(d2,2) -1

fig = plt.figure()
w = A7[0]; b = A7[1]
plt.scatter(X1[:, 0], X1[:, 1], c=d1)
plt.plot(X1[:,0], - (w[0]/w[1])*X1[:,0] - b/w[1], color = 'red', label = 'Hiperplano Ótimo')
plt.plot(X1[:,0], - (w[0]/w[1])*X1[:,0] - b/w[1] - 1/w[1], color = 'blue', label = 'Vetor Suporte')
plt.plot(X1[:,0], - (w[0]/w[1])*X1[:,0] - b/w[1] + 1/w[1], color = 'green', label = 'Vetor Suporte')
plt.xlabel('A1')
plt.ylabel('B1')
plt.title('Hiperplano ótimo encontrado através do problema dual com variável de folga para o primeiro conjunto de dados')
plt.legend()
plt.show()

# **Primeiro conjunto de dados - Sklearn**

In [ ]:
# Kernel Linear
X1, d1 = make_blobs(n_samples=20, centers=2, n_features=2,random_state=0)
d1 = 2*d1-1

clf = SVC(kernel = 'linear')
clf.fit(X1,d1)
y = clf.predict(X1)
print(accuracy_score(d1,y))

disp = DecisionBoundaryDisplay.from_estimator(clf, X1, response_method='predict', alpha=0.5)
disp.ax_.scatter(X1[:,0], X1[:,1], c=d1)
plt.xlabel('A1')
plt.ylabel('B1')

In [ ]:
# Kernel Polinomial
X1, d1 = make_blobs(n_samples=20, centers=2, n_features=2,random_state=0)
d1 = 2*d1-1

clf = SVC(kernel = 'poly', degree = 3, coef0 = 1)
clf.fit(X1,d1)
y = clf.predict(X1)
print(accuracy_score(d1,y))

disp = DecisionBoundaryDisplay.from_estimator(clf, X1, response_method='predict', alpha=0.5)
disp.ax_.scatter(X1[:,0], X1[:,1], c=d1)
plt.xlabel('A1')
plt.ylabel('B1')

In [ ]:
# Kernel Gaussiano
X1, d1 = make_blobs(n_samples=20, centers=2, n_features=2,random_state=0)
d1 = 2*d1-1

clf = SVC(kernel = 'rbf', gamma = 0.5)
clf.fit(X1,d1)
y = clf.predict(X1)
print(accuracy_score(d1,y))

disp = DecisionBoundaryDisplay.from_estimator(clf, X1, response_method='predict', alpha=0.5)
disp.ax_.scatter(X1[:,0], X1[:,1], c=d1)
plt.xlabel('A1')
plt.ylabel('B1')

In [ ]:
# Kernel Sigmoidal
X1, d1 = make_blobs(n_samples=20, centers=2, n_features=2,random_state=0)
d1 = 2*d1-1

clf = SVC(kernel = 'sigmoid', coef0 = 9.27)
clf.fit(X1,d1)
y = clf.predict(X1)
print(accuracy_score(d1,y))

disp = DecisionBoundaryDisplay.from_estimator(clf, X1, response_method='predict', alpha=0.5)
disp.ax_.scatter(X1[:,0], X1[:,1], c=d1)
plt.xlabel('A1')
plt.ylabel('B1')

# **Segundo conjunto de dados - Sklearn**

In [ ]:
# Kernel Linear
X2, d2 = make_blobs(n_samples=100, centers=2, cluster_std =4 , n_features=2,random_state=17)
d2 = 2*np.mod(d2,2) -1

clf = SVC(kernel = 'linear')
clf.fit(X2,d2)
y = clf.predict(X2)
print(accuracy_score(d2,y))

disp = DecisionBoundaryDisplay.from_estimator(clf, X2, response_method='predict', alpha=0.5)
disp.ax_.scatter(X2[:,0], X2[:,1], c=d2)
plt.xlabel('A1')
plt.ylabel('B1')

In [ ]:
# Kernel Polinomial
X2, d2 = make_blobs(n_samples=100, centers=2, cluster_std =4 , n_features=2,random_state=17)
d2 = 2*np.mod(d2,2) -1

clf = SVC(kernel = 'poly', degree = 8, coef0 = 1, gamma = 1)
clf.fit(X2,d2)
y = clf.predict(X2)
print(accuracy_score(d2,y))

disp = DecisionBoundaryDisplay.from_estimator(clf, X2, response_method='predict', alpha=0.5)
disp.ax_.scatter(X2[:,0], X2[:,1], c=d2)
plt.xlabel('A1')
plt.ylabel('B1')

In [ ]:
# Kernel Gaussiano
X2, d2 = make_blobs(n_samples=100, centers=2, cluster_std =4 , n_features=2,random_state=17)
d2 = 2*np.mod(d2,2) -1

clf = SVC(kernel = 'rbf', gamma = 0.5)
clf.fit(X2,d2)
y = clf.predict(X2)
print(accuracy_score(d2,y))

disp = DecisionBoundaryDisplay.from_estimator(clf, X2, response_method='predict', alpha=0.5)
disp.ax_.scatter(X2[:,0], X2[:,1], c=d2)
plt.xlabel('A1')
plt.ylabel('B1')

In [ ]:
# Kernel Sigmoidal
X2, d2 = make_blobs(n_samples=100, centers=2, cluster_std =4 , n_features=2,random_state=17)
d2 = 2*np.mod(d2,2) -1

clf = SVC(kernel = 'sigmoid', coef0 = 1)
clf.fit(X2,d2)
y = clf.predict(X2)
print(accuracy_score(d2,y))

disp = DecisionBoundaryDisplay.from_estimator(clf, X2, response_method='predict', alpha=0.5)
disp.ax_.scatter(X2[:,0], X2[:,1], c=d2)
plt.xlabel('A1')
plt.ylabel('B1')

# **O trabalho acaba aqui! A partir daqui os códigos não foram completamente desenvolvidos/testados**

# **Tentativa de Código Próprio - Kernel Polinomial**

In [ ]:
X1, d1 = make_blobs(n_samples=20, centers=2, n_features=2,random_state=0)
d1 = 2*d1-1

X2, d2 = make_blobs(n_samples=100, centers=2, cluster_std =4 , n_features=2,random_state=17)
d2 = 2*np.mod(d2,2) -1

def polinomialkernel(x, y, p):
    return (1 + np.dot(x, y)) ** p

def dualpoli(X,d,p):
  c = 100
  m,n = X.shape
  d = d.reshape(-1,1) * 1.0

  K = np.zeros((m, m))
  for i in range(m):
    for j in range(m):
      K[i,j] = polinomialkernel(X[i], X[j], p)

  P = cvxopt_matrix(np.outer(d,d) * K)
  q = cvxopt_matrix(-np.ones((m, 1)))
  G = cvxopt_matrix(np.vstack((np.eye(m)*-1,np.eye(m))))
  h = cvxopt_matrix(np.hstack((np.zeros(m), np.ones(m) * c)))
  A = cvxopt_matrix(d.reshape(1, -1))
  b = cvxopt_matrix(np.zeros(1))

  sol = cvxopt_solvers.qp(P, q, G, h, A, b)
  alphas = np.array(sol['x'])

  S = (alphas > 1e-4).flatten()
  dVS = d[S]
  XVS = X[S]
  alphasVS = alphas[S]
  h = len(alphasVS)

  k = np.zeros((h, h))
  for i in range(h):
    for j in range(h):
      k[i,j] = polinomialkernel(XVS[i], XVS[j], p)

  bias = np.mean(dVS - alphasVS*dVS*k)

  classificacao = np.sign(K @ (alphas*d) + bias)

  return alphas, bias, classificacao

In [ ]:
X1, d1 = make_blobs(n_samples=20, centers=2, n_features=2,random_state=0)
d1 = 2*d1-1

X2, d2 = make_blobs(n_samples=100, centers=2, cluster_std =4 , n_features=2,random_state=17)
d2 = 2*np.mod(d2,2) -1


def dualpoli(X,d,p):
  c = 0.01
  m,n = X.shape
  d = d.reshape(-1,1) * 1.0

  K = (1 + np.dot(X, X.T)) ** p

  P = cvxopt_matrix(np.outer(d,d) * K)
  q = cvxopt_matrix(-np.ones((m, 1)))
  G = cvxopt_matrix(np.vstack((np.eye(m)*-1,np.eye(m))))
  h = cvxopt_matrix(np.hstack((np.zeros(m), np.ones(m) * c)))
  A = cvxopt_matrix(d.reshape(1, -1))
  b = cvxopt_matrix(np.zeros(1))

  sol = cvxopt_solvers.qp(P, q, G, h, A, b)
  alphas = np.array(sol['x'])

  S = (alphas > 1e-4).flatten()
  dVS = d[S]
  XVS = X[S]
  alphasVS = alphas[S]
  h = len(alphasVS)

  k = (1 + np.dot(XVS, XVS.T)) ** p

  bias = np.mean(dVS - alphasVS*dVS*k)

  classificacao = np.sign(K @ (alphas*d) + bias)

  return alphas, bias, classificacao

# **Tentativa de Código Próprio - Kernel Gaussiano**

In [ ]:
X1, d1 = make_blobs(n_samples=20, centers=2, n_features=2,random_state=0)
d1 = 2*d1-1

X2, d2 = make_blobs(n_samples=100, centers=2, cluster_std =4 , n_features=2,random_state=17)
d2 = 2*np.mod(d2,2) -1

def gaussiankernel(x, y, sigma):
    return np.exp(-np.linalg.norm(x-y)**2 / (2 * (sigma ** 2)))

def dualgau(X,d,sigma):
  c = 0.01
  m,n = X.shape
  d = d.reshape(-1,1) * 1.0

  K = np.zeros((m, m))
  for i in range(m):
    for j in range(m):
      K[i,j] = gaussiankernel(X[i], X[j], sigma)

  P = cvxopt_matrix(np.outer(d,d) * K)
  q = cvxopt_matrix(-np.ones((m, 1)))
  G = cvxopt_matrix(np.vstack((np.eye(m)*-1,np.eye(m))))
  h = cvxopt_matrix(np.hstack((np.zeros(m), np.ones(m) * c)))
  A = cvxopt_matrix(d.reshape(1, -1))
  b = cvxopt_matrix(np.zeros(1))

  sol = cvxopt_solvers.qp(P, q, G, h, A, b)
  alphas = np.array(sol['x'])

  S = (alphas > 1e-4).flatten()
  dVS = d[S]
  XVS = X[S]
  alphasVS = alphas[S]
  h = len(alphasVS)

  k1 = np.zeros((h, h))
  for i in range(h):
    for j in range(h):
      k1[i,j] = gaussiankernel(XVS[i], XVS[j], sigma)

  bias = np.mean(dVS - alphasVS*dVS*k1)

  classificacao = np.sign(K @ (alphas*d) + bias)

  return alphas, bias, classificacao

A11= dualgau(X1,d1,1)
A12= dualgau(X2,d2,1)

print(A11, A12)

In [ ]:
# Testando Acurácia
f = A12[2].flatten() == d2.flatten()
print(f)

acc = np.sum(f)/len(f)
print(acc)